In [1]:
import os
import glob
import warnings
import re
from astropy.io import fits
from PIL import Image
import numpy as np

# Define user parameters
User_myFolder = r'C:\MIT\COSMOS\Matlab\Images'
User_fits = 1
User_figures = r'C:\Users\Jill\Desktop\COSMOS Play\Matlab\Results'
User_save_figures = 1

# Check to make sure that folder actually exists. Warn user if it doesn't.
if not os.path.isdir(User_myFolder):
    error_message = f'Error: The following folder does not exist:\n{User_myFolder}'
    warnings.warn(error_message)
    raise FileNotFoundError(error_message)

# Function to find keywords in FITS header
def find_keywords(keywords, compare_word):
    indices = []
    for i, keyword in enumerate(keywords):
        if re.search(compare_word, keyword, re.IGNORECASE):
            indices.append(i)
    return indices

# Function to extract text between quotes
def extract_text_between_quotes(input_string):
    matches = re.findall(r'"([^"]*)"', input_string)
    if matches:
        return matches[0]
    else:
        return ''

# Initialize the stack list
stack = []

# Get a list of all files in the folder with the desired file name pattern.
if User_fits == 1:
    file_pattern = os.path.join(User_myFolder, '*.fits')
    the_files = glob.glob(file_pattern)

    for file in the_files:
        base_file_name = os.path.basename(file)
        full_file_name = os.path.join(User_myFolder, base_file_name)
        print(f'Now reading {full_file_name}')
        
        # Read FITS file
        hdul = fits.open(full_file_name)
        info = hdul.info()
        imagestack = hdul[0].data
        
        # Append to stack
        stack.append({
            'baseFileName': base_file_name,
            'fullFileName': full_file_name,
            'info': info,
            'imagestack': imagestack
        })
        
        # Close the FITS file
        hdul.close()
else:
    file_pattern = os.path.join(User_myFolder, '*.tif')
    the_files_tif = glob.glob(file_pattern)

    for file in the_files_tif:
        base_file_name = os.path.basename(file)
        full_file_name = os.path.join(User_myFolder, base_file_name)
        print(f'Now reading {full_file_name}')
        
        # Read TIFF file
        im = Image.open(full_file_name)
        imagestack = []
        try:
            while True:
                imagestack.append(np.array(im))
                im.seek(im.tell() + 1)
        except EOFError:
            pass
        
        imagestack = np.stack(imagestack, axis=-1)
        
        # Append to stack
        stack.append({
            'baseFileName': base_file_name,
            'fullFileName': full_file_name,
            'imagestack': imagestack
        })

# Keywords to find
cosmoskeywords = ['DATE', 'DATE-OBS', 'BITPIX']
fieldnames = ['date', 'time_stamp_acq', 'bit_per_pix']

# Extract and store keywords from FITS headers
for file_data in stack:
    if User_fits == 1:
        hdul = fits.open(file_data['fullFileName'])
        primary_header = hdul[0].header
        
        for keyword, fieldname in zip(cosmoskeywords, fieldnames):
            primary_keywords = list(primary_header.keys())
            primary_indices = find_keywords(primary_keywords, keyword)
            if primary_indices:
                index = primary_indices[0]
                file_data[fieldname] = primary_header[primary_keywords[index]]
            else:
                file_data[fieldname] = None
        
        hdul.close()
    else:
        # Handle TIFF files if needed
        pass

# Keywords to find for COSMOS data
cosmoskeywords = ['SENSOR INFORMATION SENSORNAME', 'ACTIVEAREA HEIGHT', 'ACTIVEAREA WIDTH', 'ACTIVEAREA BOTTOMMARGIN',
                  'ACTIVEAREA LEFTMARGIN', 'ACTIVEAREA RIGHTMARGIN', 'PIXEL WIDTH', 'ADC BITDEPTH', 'ANALOGGAIN',
                  'ADC QUALITY', 'CORRECTPIXEL', 'FRAMERATE', 'FRAMESTOSTORE', 'BASEFILENAME', 'TEMPERATURE READING',
                  'READOUTCONTROL MODE', 'SHUTTERTIMING EXPOSURETIME']
fieldnames = ['sensor_info', 'active_height', 'active_width', 'bottom_margin', 'leftmargin', 'rightmargin',
              'pixel_size_um', 'adc_bit', 'analog_gain', 'adc_speed', 'corrected', 'fps', 'frame_num', 'filename',
              'cam_temp_Cel', 'shutter', 'exposure_ms']

# Extract and store COSMOS keywords from FITS headers
for file_data in stack:
    if User_fits == 1:
        hdul = fits.open(file_data['fullFileName'])
        primary_header = hdul[0].header
        
        for keyword, fieldname in zip(cosmoskeywords, fieldnames):
            primary_keywords = list(primary_header.keys())
            primary_indices = find_keywords(primary_keywords, keyword)
            if primary_indices:
                index = primary_indices[0]
                extracted_text = extract_text_between_quotes(primary_header.comments[primary_keywords[index]])
                file_data[fieldname] = extracted_text
            else:
                file_data[fieldname] = None
        
        hdul.close()
    else:
        # Handle TIFF files if needed
        pass

# Display the stack dictionary
for file_data in stack:
    print(file_data)


Now reading C:\MIT\COSMOS\Matlab\Images\test.fits
Filename: C:\MIT\COSMOS\Matlab\Images\test.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     105   (8120, 8120, 50)   int16 (rescales to uint16)   
{'baseFileName': 'test.fits', 'fullFileName': 'C:\\MIT\\COSMOS\\Matlab\\Images\\test.fits', 'info': None, 'imagestack': array([[[137, 147, 150, ...,  87,  93,  99],
        [140, 152, 151, ..., 101, 106, 104],
        [135, 154, 151, ...,  98, 106, 108],
        ...,
        [113, 126, 118, ...,  95, 100, 104],
        [105, 117, 120, ...,  96, 102, 103],
        [105, 116, 120, ...,  96, 106, 101]],

       [[134, 143, 148, ...,  88,  96,  96],
        [135, 145, 143, ...,  99, 105, 104],
        [137, 159, 153, ..., 101, 105, 109],
        ...,
        [110, 124, 118, ...,  98,  99, 102],
        [104, 119, 123, ...,  97, 104, 104],
        [105, 117, 123, ...,  98, 104,  99]],

       [[136, 147, 151, ...,  86,  92,  99],
        [138, 